# 資料預處理

In [1]:
import os
os.chdir("C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code")
import sys
sys.path.append("C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code")

from Get_file import Get_file   # return dirpath, dirnames, filenames
from FFT import FFT, PlotDataOnTimeDomain
from BandPassFilter import BandPassFilter, PlotDataOnFreqDomain, BandPassListFilter
import pyedflib
import numpy as np
import datetime

import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA, PCA

%matplotlib inline

data_path = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\database"  # EEG Data direction
BandPassICAFigureSavingDirection = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code\\Fig"
SaveFFTAllDataDirection = "C:\\Users\\zxpay\\Desktop\\BME_Senior\\EEG_CLF\\Emotion\\Code\\Fig\\FFTFig_AllData"

# Setting parameters
SampleFrequency = 128
DEBUG = True
LabelSize = 40
LowerFreqCut = 1
HigherFreqCut = 50
BandPassOrder = 5
channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
SAVE_FIG = False


### 讀取檔案

In [22]:
#Four Lables
normal_label = []   # 0
scare_label = []    # 1
touch_label = []    # 2
laugh_label = []    # 3

#Four Signal
normal_signal = []
scare_signal = []
touch_signal = []
laugh_signal = []

signal = []
labels = []

_,data_files,_ = Get_file(data_path)
if data_files == []:
    raise("Not found any files ! check your data_path")
else:
    print('Get data from database ...')
    for data in data_files:
        _,_,file_names = Get_file(data_path+'\\'+data)
        for fn in file_names:
            if fn.split('.')[1] == 'edf':
                fedf = pyedflib.EdfReader(data_path+'\\'+data+'\\'+fn)
                n = fedf.signals_in_file
                signal_labels = np.asarray(fedf.getSignalLabels())
                sigbufs = np.zeros((n, fedf.getNSamples()[0]))
                for i in np.arange(n):
                    sigbufs[i, :] = fedf.readSignal(i)

                if 'normal' in fn:
                    normal_label.append(0)
                    normal_signal.append(sigbufs)
                elif 'scare' in fn:
                    scare_label.append(1)
                    scare_signal.append(sigbufs)
                elif 'touch' in fn:
                    touch_label.append(2)
                    touch_signal.append(sigbufs)
                elif 'laugh' in fn:
                    laugh_label.append(3)
                    laugh_signal.append(sigbufs)
                    
                if DEBUG:
                    print(sigbufs.shape)

''' Labels --> 40 channels
['COUNTER' 'INTERPOLATED' 'AF3' 'F7' 'F3' 'FC5' 'T7' 'P7' 'O1' 'O2' 'P8'
 'T8' 'FC6' 'F4' 'F8' 'AF4' 'RAW_CQ' 'GYROX' 'GYROY' 'MARKER'
 'MARKER_HARDWARE' 'SYNC' 'TIME_STAMP_s' 'TIME_STAMP_ms' 'CQ_AF3' 'CQ_F7'
 'CQ_F3' 'CQ_FC5' 'CQ_T7' 'CQ_P7' 'CQ_O1' 'CQ_O2' 'CQ_P8' 'CQ_T8' 'CQ_FC6'
 'CQ_F4' 'CQ_F8' 'CQ_AF4' 'CQ_CMS' 'CQ_DRL']
'''
ChannelDict = {
    0 : 'COUNTER', 1 : 'INTERPOLATED', 2 : 'AF3', 3 : 'F7',
    4 : 'F3', 5 : 'FC5', 6 : 'T7', 7 : 'P7',
    8 : 'O1', 9 : 'O2', 10 : 'P8', 11 : 'T8',
    12 : 'FC6', 13 : 'F4', 14 : 'F8', 15 : 'AF4',
    16 : 'RAW_CQ', 17 : 'GYROX', 18 : 'GYROY', 19 : 'MARKER',
    20 : 'MARKER_HARDWARE', 21 : 'SYNC', 22 : 'TIME_STAMP_s', 23 : 'TIME_STAMP_ms',
    24 : 'CQ_AF3', 25 : 'CQ_F7', 26 : 'CQ_F3', 27 : 'CQ_FC5',
    28 : 'CQ_T7', 29 : 'CQ_P7', 30 : 'CQ_O1', 31 : 'CQ_O2',
    32 : 'CQ_P8', 33 : 'CQ_T8', 34 : 'CQ_FC6', 35 : 'CQ_F4',
    36 : 'CQ_F8', 37 : 'CQ_AF4', 38 : 'CQ_CMS', 39 : 'CQ_DRL',
}

print('Okay')
if DEBUG:
    print('normal signal length : ', len(normal_signal))
    print('scare signal length : ', len(scare_signal))
    print('touch signal length : ', len(touch_signal))
    print('laugh signal length : ', len(laugh_signal))


Get data from database ...
(40, 38400)
(40, 105344)
(40, 38400)
(40, 105344)
(40, 179328)
(40, 309760)
(40, 38400)
(40, 105728)
(40, 158976)
(40, 58112)
(40, 50944)
(40, 33152)
(40, 3968)
(40, 38400)
(40, 105344)
(40, 178944)
(40, 308992)
(40, 38400)
(40, 105728)
(40, 179840)
(40, 237312)
(40, 38400)
(40, 105600)
(40, 179584)
(40, 38400)
(40, 178944)
(40, 308864)
(40, 38400)
(40, 105344)
(40, 179072)
(40, 83328)
(40, 117632)
(40, 107392)
(40, 38400)
(40, 308992)
(40, 38400)
(40, 179072)
(40, 105344)
(40, 308736)
(40, 38400)
(40, 313472)
(40, 38400)
(40, 105344)
(40, 38400)
(40, 105344)
(40, 38528)
(40, 179200)
(40, 308736)
(40, 38400)
(40, 179328)
(40, 38400)
(40, 105344)
Okay
normal signal length :  16
scare signal length :  11
touch signal length :  9
laugh signal length :  16


In [3]:
def GrabEmotion(Signal, TimeSecondsi, TimeSecondsf):
    pass

In [4]:
def CheckListTheSameSize(ListData):
    di = 0
    dsum = 0
    for i in range(len(ListData)):
        if i == 0:
            pass
        else:
            dsum += (ListData[i].shape == ListData[i-1].shape)
            
    if np.sum(dsum) != (len(ListData)-1):
        return False
    else:
        return True
    
def CalculateZeros(NumpyArray):
    LogicWithZeros = (NumpyArray == 0)
    return np.sum(LogicWithZeros)

def PlotFreqencyFigure(FreqList, FFTData, FreqL, FreqH, YlimL=None, YlimH=None, title=None, FigSize=(12,8)):
    plt.figure(figsize=FigSize)
    plt.plot(FreqList, FFTData, '*-')
    plt.xlabel('Freq(Hz)')
    plt.ylabel('micro volt (mv)')
    plt.xlim(FreqL, FreqH)
    if YlimL is None and YlimH is None:
        pass
    else:
        plt.ylim(YlimL, YlimH)
    if title is not None:
        plt.title(title)
        
    else:
        plt.title('FreqencyFigure')
    plt.grid(True)
    plt.show()

### Analyze steps : Filters ==> ICA ==> FFT ==> Build Model

In [5]:
# Filter, Each Data 128*5 second to as a period
# Return Data Shape : (DataNumbers, Channels, Features)
def BandPassSecond(SignalListData, LowerFreqCut, HigherFreq, fs, order=5, PlotFlag=False):
    channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
    SamplesPerSeconds = fs
    SignalData = np.zeros([len(SignalListData), len(channel_we_use), 64], dtype=np.float32)
    for i in range(len(SignalListData)):
        LabelNum = SignalListData[i].shape[0]
        FeaturesNum = SignalListData[i].shape[1]
        TimesToSplit = int(FeaturesNum/SamplesPerSeconds)
        print('Features Numbers : ', FeaturesNum)
        print('Times to Split Numbers :', TimesToSplit)
        ChCNT = 0
        for ch in range(LabelNum):
            if ch in channel_we_use:
                if ch in channel_we_use:
                    ChannelBuffer = np.zeros([TimesToSplit, SamplesPerSeconds], dtype=np.float32)
                    ChannelMean = np.zeros(SamplesPerSeconds, dtype=np.float32)
                    Times = 0
                    for t in range(TimesToSplit):
                        try:
                            ChannelBuffer[t][:] = BandPassFilter(SignalListData[i][ch][Times:Times+SamplesPerSeconds],
                                                                 LowerFreqCut, HigherFreqCut, fs, order=order)
                            Times += SamplesPerSeconds
                        except Exception as e:
                            pass
                    ChannelMean = np.mean(ChannelBuffer, axis=0)
                    SignalFreqX, ChannelMean = FFT(ChannelMean.reshape(-1), SampleFrequency, data_length=128)
                    SignalData[i][ChCNT][:] = ChannelMean
                    if PlotFlag:
                        PlotFreqencyFigure(SignalFreqX, SignalData[i][ChCNT][:], 8, 30, 0, 200)
                ChCNT += 1
    print()
    if CalculateZeros(SignalData) == 0:
        return SignalFreqX, SignalData
    else:
        print("Warning: Have Zero Values in the Numpy array !!!")
        return SignalFreqX, SignalData

In [6]:
# Filter, Each Data 128*5 second to as a period
# Return Data Shape : (DataNumbers, Channels, Features)
def BandPassAllData(SignalListData, LowerFreqCut, HigherFreq, fs, order=5, PlotFlag=False):
    channel_we_use = [2,3,4,5,6,7,8,9,10,11,12,13,14,15]   # 0~39 共40, 14 channels we use
    SamplesPerSeconds = fs
    SignalData = np.zeros([len(SignalListData), len(channel_we_use), 64], dtype=np.float32)
    for i in range(len(SignalListData)):
        LabelNum = SignalListData[i].shape[0]
        FeaturesNum = SignalListData[i].shape[1]
        print('Features Numbers : ', FeaturesNum)
        ChCNT = 0
        for ch in range(LabelNum):
            if ch in channel_we_use:
                if ch in channel_we_use:
                    SignalBeforeFFT = SignalListData[i][ch][:]
                    SignalFreqX, SignalData[i][ChCNT][:] = FFT(SignalBeforeFFT, SampleFrequency, data_length=128)
                    if PlotFlag:
                        PlotFreqencyFigure(SignalFreqX, SignalData[i][ChCNT][:], 8, 30, 0, 5)
                    ChCNT += 1
    print()
    if CalculateZeros(SignalData) == 0:
        return SignalFreqX, SignalData
    else:
        print("Warning: Have Zero Values in the Numpy array !!!")
        return SignalFreqX, SignalData

In [7]:


SignalFreqX, NormalSignalArray = BandPassSecond(normal_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, ScareSignalArray = BandPassSecond(scare_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, TouchSignalArray = BandPassSecond(touch_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)
SignalFreqX, LaughSignalArray = BandPassSecond(laugh_signal, LowerFreqCut, HigherFreqCut, SampleFrequency, order=BandPassOrder, PlotFlag=False)

if DEBUG:
    print('Normal Data shape:', NormalSignalArray.shape)
    print('Scare Data shape:', ScareSignalArray.shape)
    print('Touch Data shape:', TouchSignalArray.shape)
    print('Laugh Data shape:', LaughSignalArray.shape)

Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38528
Times to Split Numbers : 301
Features Numbers :  38400
Times to Split Numbers : 300
Features Numbers :  38400
Times to Split Numbers : 300

Features Numbers :  105344
Times to Split Numbers : 823
Features Numbers :  105344
Times to Split Numbers : 823
Feature

In [8]:
# # Channel we choose and get the channel data
# def FFTChannelData(SignalArray, Channel, SAVE_FIG=False, name='FFTFreqFig'):
#     SignalNums = SignalArray.shape[0]
#     SignalChannelNums = SignalArray.shape[1]
#     SignalFeaturesNums = SignalArray.shape[2]
#     SignalChannelData = np.zeros([SignalNums, SignalFeaturesNums])
#     a = 0
#     b = 0
#     for i in range (SignalNums):
#         b = NormalSignalArray[i,Channel,:].reshape(-1)
#         SignalChannelData[i,Channel,:] = SignalArray[i,Channel,:]
#         if SAVE_FIG == True:
#             os.chdir(SaveFFTAllDataDirection)
#             fig = plt.gcf()   # Get Current Figure, 不加這行，save figire 時會存成白色
#             PlotFreqencyFigure(SignalFreqX,b,0,63,0,10)
#             FigName = name + '_%d' % (i+1) + '_ch%d' % Channel  + '.png'
#             fig.savefig(FigName)
# FFTChannelData(NormalSignalArray, 0, SAVE_FIG=False, name='Normal')

### Collect Channel one data to be the Training Data

In [9]:
NormalNum = NormalSignalArray.shape[0]
ScareNum = ScareSignalArray.shape[0]
TouchNum = TouchSignalArray.shape[0]
LaughNum = LaughSignalArray.shape[0]
# Create the y label
NormalLabel = np.ones(NormalNum, dtype=np.float32) * 1
ScareLabel = np.ones(ScareNum, dtype=np.float32) * 2
TouchLabel = np.ones(TouchNum, dtype=np.float32) * 3
LaughLabel = np.ones(LaughNum, dtype=np.float32) * 4
for i in range(len(channel_we_use)):
    if i == 0:
        TrainingLabel = np.hstack((NormalLabel, ScareLabel, TouchLabel, LaughLabel[:]))
    else:
        y = np.hstack((NormalLabel, ScareLabel, TouchLabel, LaughLabel[:]))
        TrainingLabel = np.hstack((y, TrainingLabel))
print('Training Lable Shape : ', TrainingLabel.shape)

Training Lable Shape :  (728,)


In [10]:
# Get channel 1 data as Training data
# [8:31] is alpha and beta frequency domain
# channel = 12
# NormalChannel1 = NormalSignalArray[:,channel,8:31]
# ScareChannel1 = ScareSignalArray[:,channel,8:31]
# TouchChannel1 = TouchSignalArray[:,channel,8:31]
# LaughChannel1 = LaughSignalArray[:,channel,8:31]
# TrainingData = np.vstack((NormalChannel1, ScareChannel1, TouchChannel1, LaughChannel1))
# print('Training Data Shape : ', TrainingData.shape)

In [11]:
'''
normal signal length :  16
scare signal length :  11
touch signal length :  9
laugh signal length :  16
'''
for channel in range(len(channel_we_use)):
    if channel == 0:
        TrainingData = np.vstack((NormalSignalArray[:,channel,8:31].reshape([16,23]),
                                  ScareSignalArray[:,channel,8:31].reshape([11,23]),
                                  TouchSignalArray[:,channel,8:31].reshape([9,23]),
                                  LaughSignalArray[:,channel,8:31].reshape([16,23])))
    else:
        X = np.vstack((NormalSignalArray[:,channel,8:31].reshape([16,23]),
                                  ScareSignalArray[:,channel,8:31].reshape([11,23]),
                                  TouchSignalArray[:,channel,8:31].reshape([9,23]),
                                  LaughSignalArray[:,channel,8:31].reshape([16,23])))
        TrainingData = np.vstack((TrainingData, X))
    print(TrainingData.shape)
print('Training Data Shape : ', TrainingData.shape)

(52, 23)
(104, 23)
(156, 23)
(208, 23)
(260, 23)
(312, 23)
(364, 23)
(416, 23)
(468, 23)
(520, 23)
(572, 23)
(624, 23)
(676, 23)
(728, 23)
Training Data Shape :  (728, 23)


### Build Training Model

In [28]:
from Danny_ML_CLF import Danny_ML_CLF
clf = Danny_ML_CLF()
train_X, test_X, train_y, test_y = clf.Split_data(TrainingData, TrainingLabel, test_size=0.3, Standard=True)
clf.Fit_value(train_X, train_y)
clf.Train()
clf.Report(test_X, test_y, [1,2,3,4], show_cm=False)
clf.Report2txt('EEG_Report2.txt')
print('Report Okay !\n\n')
clf.Score()

C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Report Okay !


SVM Score :  0.4885844748858447
Tree Score :  0.5662100456621004
Bayes Score :  0.3561643835616438
KNN Score :  0.680365296803653
XGB Score :  0.634703196347032
Stacking Score :  0.6529680365296804
Voting Score :  0.6255707762557078
Bagging Score :  0.680365296803653
RF Score :  0.680365296803653
Adaboost Score :  0.5570776255707762


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [14]:
NormalSignalArray.shape

(16, 14, 64)

In [33]:
for i in range(len(channel_we_use)):
    Nor = NormalSignalArray[:,i,8:31].reshape(NormalSignalArray.shape[0], 23)
    Sca = ScareSignalArray[:,i,8:31].reshape(ScareSignalArray.shape[0], 23)
    Tou = TouchSignalArray[:,i,8:31].reshape(TouchSignalArray.shape[0], 23)
    Lau = LaughSignalArray[:,i,8:31].reshape(LaughSignalArray.shape[0], 23)
    y1 = np.ones([NormalSignalArray.shape[0]], dtype=np.float32) * 1
    y2 = np.ones([ScareSignalArray.shape[0]], dtype=np.float32) * 2
    y3 = np.ones([TouchSignalArray.shape[0]], dtype=np.float32) * 3
    y4 = np.ones([LaughSignalArray.shape[0]], dtype=np.float32) * 4
    X = np.vstack((Nor, Sca, Tou, Lau))
    y = np.hstack((y1,y2,y3,y4))

    clf = Danny_ML_CLF()
    train_X, test_X, train_y, test_y = clf.Split_data(X, y, test_size=0.3, Standard=True)
    print(train_X.shape)
    print(train_y.shape)   
    clf.Fit_value(train_X, train_y)
    clf.Train()
    clf.Report(test_X, test_y, [1,2,3,4], show_cm=False)
    clf.Report2txt(ChannelDict[channel_we_use[i]] + '.txt')
    print('Report Okay !\n\n')
    clf.Score()

(36, 23)
(36,)


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

Report Okay !


SVM Score :  0.5
Tree Score :  0.5
Bayes Score :  0.25
KNN Score :  0.5625
XGB Score :  0.5
Stacking Score :  0.5
Voting Score :  0.3125
Bagging Score :  0.625
RF Score :  0.6875
Adaboost Score :  0.375
(36, 23)
(36,)


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

Report Okay !


SVM Score :  0.125
Tree Score :  0.125
Bayes Score :  0.3125
KNN Score :  0.125
XGB Score :  0.1875
Stacking Score :  0.125
Voting Score :  0.125
Bagging Score :  0.125
RF Score :  0.125
Adaboost Score :  0.25
(36, 23)
(36,)


...

C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

Report Okay !


SVM Score :  0.5
Tree Score :  0.375
Bayes Score :  0.4375
KNN Score :  0.375
XGB Score :  0.25
Stacking Score :  0.3125
Voting Score :  0.4375
Bagging Score :  0.3125
RF Score :  0.4375
Adaboost Score :  0.4375
(36, 23)
(36,)


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Report Okay !


SVM Score :  0.375
Tree Score :  0.1875
Bayes Score :  0.125
KNN Score :  0.5
XGB Score :  0.375
Stacking Score :  0.3125
Voting Score :  0.25
Bagging Score :  0.375
RF Score :  0.375
Adaboost Score :  0.1875


C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\zxpay\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [32]:
print(X.shape)
print(y.shape)

(52, 23)
(52,)
